In [1]:
# Module Importations
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import keras
import numpy as np
import time

print('sklearn Version', sklearn.__version__)
print('keras Version', keras.__version__)

Using TensorFlow backend.
keras Version 2.3.1


In [2]:
# Custom Module Imports
from src.data import load_data
from src.data import split_data
from src.models import model_evaluation
from src.models import keras_helpers 

In [3]:
# Load Dataset
original_dataset = load_data.load_motor_data()

In [4]:
# Split Data into Training & Evaluation Sets
train_set, eval_set = split_data.split_train_test(original_dataset, 0.8)

In [5]:
# Drop Profile ID (Cleaning)
train_set = train_set.drop("profile_id", axis = 1)
eval_set = eval_set.drop("profile_id", axis = 1)

In [6]:
# Create Torque Target Dataset
torque_train_data = train_set.drop("torque", axis = 1)
torque_label_data = train_set["torque"].copy()

torque_eval_data = eval_set.drop("torque", axis = 1)
torque_eval_label = eval_set["torque"].copy()

In [7]:
# Create Evaluation Metric (Range)

# Calculate arithmetic range of torque readings
torque_range = original_dataset["torque"].max() - original_dataset["torque"].min()

print("Torque range:", torque_range)

Torque range: 6.3629233


In [11]:
# Create Train / Test Arrays
X_train, X_test, y_train, y_test = train_test_split(torque_train_data, torque_label_data, test_size = 0.2, random_state = 0)

print(X_train.shape)

(159691, 11)


In [ ]:
# Multilayer Perceptron NN (Target - Torque)

# Clear existing models
keras.backend.clear_session()

# Investigate impact of parameter distribution
param_distribs = {
    "n_hidden": [1, 4, 8],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocals(3e-4, 3e-2)
    }

# Build model
wrapped_model = keras_helpers.wrap_model()

# Initialise random search
rnd_search_cv = RandomizedSearchCV(wrapped_model, param_distribs, n_iter = 10, cv = 3)

# Name model
model_type = "MLP-Torque"
model_id = keras_helpers.name_model(model_type)
filepath_full = keras_helpers.make_save_string(model_id)

# Implement save and earlystop callbacks
earlystop_cb = keras.callbacks.EarlyStopping(patience = 5)
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = filepath_full, save_best_only = True)

# Train model
rnd_search_cv.fit(X_train, y_train, epochs = 100, validation_data = (X_test, y_test), callbacks = [checkpoint_cb, earlystop_cb])